In [1]:
import urllib.request
from bs4 import BeautifulSoup
import warnings
import datetime
warnings.simplefilter("ignore")
from datetime import datetime

from collections import namedtuple
import pickle
import pandas as pd

from IPython.display import clear_output
from selenium import webdriver
from time import sleep
import json
from math import radians, cos, sin, asin, sqrt
import copy

import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import multiprocessing
import numpy as np
import pyodbc
from sqlalchemy import create_engine
import time

In [20]:
atm = pickle.load(open('atm_target.data','rb'))

In [21]:
atm

,atm_acqbin,atm_caid,atm_terminal,full_address,traget,log_traget,address,city,address_rus,lat,long,adress
488,418388,ATM 0007,S1AM0007,SOVETSKAYA 47A SARANSK,46290000.0,17.650437,SOVETSKAYA 47A,SARANSK,,54.181247,45.182941,"Советская улица, 47А, Саранск, Республика Морд..."
489,418388,ATM 0023,S1AM0023,SUVOROVA 44 IVANOVO,233770000.0,19.269848,SUVOROVA 44,IVANOVO,,56.994068,41.007096,"улица Суворова, 44, Иваново, Россия, 153012"
490,418388,ATM 0024,S1AM0024,NEVSKOGO 7 KURSK,252395000.0,19.346506,NEVSKOGO 7,KURSK,,51.725748,36.187077,"улица Александра Невского, 7, Курск, Россия, 3..."
491,418388,ATM 0042,S1AM0042,KIROVA 9A KALUGA,230980000.0,19.257842,KIROVA 9A,KALUGA,,54.515920,36.249285,"улица Кирова, 9А, Калуга, Россия, 248001"
492,418388,ATM 0070,S1AM0070,RIDZINSKOGO 24 YAKUTSK,290810000.0,19.488181,RIDZINSKOGO 24,YAKUTSK,,62.047895,129.718748,"улица Рыдзинского, 24, Якутск, Республика Саха..."
493,418388,ATM 0076,S1AM0076,ZAPOLNAYA 47 KURSK,22220000.0,16.916503,ZAPOLNAYA 47,KURSK,,51.742275,36.149914,"Запольная улица, 47, Курск, Россия, 305040"
494,418388,ATM 0079,S1AM0079,PIONERSKAYA 5 BARYSH,47510000.0,17.676451,PIONERSKAYA 5,BARYSH,,53.653095,47.105857,"Пионерская улица, 5, Барыш, Ульяновская област..."
495,418388,ATM 0082,S1AM0082,ESENINA 9 RYAZAN,93900000.0,18.357741,ESENINA 9,RYAZAN,,NaN,NaN,NaN
496,418388,ATM 0085,S1AM0085,BEKESHSKAYA 39 PENZA,304230000.0,19.533295,BEKESHSKAYA 39,PENZA,,NaN,NaN,NaN
497,418388,ATM 0086,S1AM0086,KOMSOMOL. PLAZA 1 KAMENKA,381130000.0,19.758651,KOMSOMOL. PLAZA 1,KAMENKA,,NaN,NaN,NaN


In [4]:
atm.columns  = ['atm_acqbin','atm_caid','atm_terminal','full_address','traget', 'log_traget']

In [5]:
atm['address'] = atm.full_address.apply(lambda x: x[:23].strip())

In [6]:
atm['city'] = atm.full_address.apply(lambda x: x[23:].strip()).apply(lambda x: x[:-2] if x.endswith(' G') else x)

In [7]:
def transform(x):
    dictionary = {"MOSKVA": "MOSCOW",
                  "SANKT-PETERBU": "SAINT PETERSBURG",
                  'S.-PETERBURG':  "SAINT PETERSBURG",
                  'ST-PETERSBURG': "SAINT PETERSBURG",
                  'ST PETERSBURG': "SAINT PETERSBURG"}
    for key, value in dictionary.items():
        x = x.replace(key, value)
    return x

In [8]:
atm.city = atm.city.map(transform)

In [22]:
atm['address_rus'] = ''
atm['lat'] = np.nan
atm['long'] = np.nan

In [23]:
del atm['adress']

In [10]:
to_replace = {
     'AV.': ' PROSPECT ',
     'AVE': ' PROSPECT ', 
     'PR-KT': ' PROSPECT ',
     'PR-T': ' PROSPECT ',
     'STR.': ' ULITSA ',
     ' ST ': ' ULITSA ',
     'UL.' : ' ULITSA ',
     'H/W.': ' SHOSSE ',
     ', LIT. ': '',
     'D.': '',
     'VLAD. ':'',
     ', KORP. ': '/',
     ' DZERZHINSK':  ' DZERZHINSKOGO ',
     '.': '. ',
     ',': ', ',
}

In [11]:
for i in atm.index:
    for key, value in to_replace.items():
        atm.at[i,'address'] =  atm.at[i,'address'].replace(key, value)

In [19]:
for i in atm.index:
    atm.at[i,'address'] = atm.at[i,'address'][4:] if atm.at[i,'address'][:4] == 'ABB ' else atm.at[i,'address']

# Парсер открытой информации из HeadHanter

In [25]:
def log_progress(sequence, every=10):
    for index, item in enumerate(sequence):
        if index % every == 0:
            clear_output()
            print(index)
        yield item

In [36]:
def parsing(atm, start):
    driver = webdriver.Chrome('chromedriver.exe')
    url = 'https://yandex.ru/maps/213/moscow/?ll=37.530858%2C55.640315&z=12&mode=search&text='
    for i, atmi in log_progress(list(atm.iterrows())[start:]):
        urli = url + ' '.join([atmi.address, atmi.city]).replace(' ', '%20')
        driver.get(urli)
        try: 
            element = driver.find_element_by_xpath("//div[@class='clipboard__action-wrapper _inline']")
            atm.at[i, 'lat'],  atm.at[i, 'long'] = list(map(float, element.text.split(',')))

            addr_part1 = driver.find_element_by_xpath("//div[@class='card-title-view__title']")
            addr_part2 = driver.find_element_by_xpath("//div[@class='toponym-card-view__description']")
            atm.at[i, 'address_rus'] = addr_part1.text + ', ' + addr_part2.text 
        except:
            pass
        time.sleep(0.3)

In [30]:
pickle.dump(atm, open('atm_target_v2.data','wb'))

In [53]:
pickle.dump(atm, open('atm_target_no_rba.data','wb'))

In [51]:
parsing(atm, 2010 + 1240 + 850 + 1190 + 4530 + 1940)

150


In [60]:
atm.full_address.apply(lambda x: x.startswith('RBA'))

1314

In [62]:
ros_atm = atm[atm.full_address.apply(lambda x: x.startswith('RBA'))]